<a href="https://colab.research.google.com/github/mohdraavi/Hotel-Booking-Analysis/blob/main/EDA_Hotel_Booking_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - Capstone Project-1 EDA Hotel Booking Analysis.



##### **Project Type**    - EDA
##### **Contribution**    - Individual
##### **Project By Mohd Ravi -**


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

https://github.com/mohdraavi/Hotel-Booking-Analysis

# **Problem Statement**


Have you ever wondered when the best time of year to book a hotel room is? Or the optimal length of stay in order to get the best daily rate? What if you wanted to predict whether or not a hotel was likely to receive a disproportionately high number of special requests? This hotel booking dataset can help you explore those questions! This data set contains booking information for a city hotel and a resort hotel, and includes information such as when the booking was made, length of stay, the number of adults, children, and/or babies, and the number of available parking spaces, among other things. All personally identifying information has been removed from the data.

#### **Define Your Business Objective?**

 Explore and analyse the data to discover important factors that govern the bookings

### Import Libraries

In [ ]:
#Mount the Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import Libraries
import numpy as np                          #Numerical Computation
import pandas as pd                         #Data Manipulation
import matplotlib.pyplot as plt             #visulize with plots
%matplotlib inline 
import seaborn as  sns                      #Data Visulization


### Dataset Loading

In [ ]:
# Load Dataset
df = pd.read_csv('/content/drive/MyDrive/Hotel Bookings.csv')

### Dataset First View

In [ ]:
# Dataset First Look
#initial five rows 
df.head()

In [ ]:
#last five rows 
df.tail()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df.shape

there are 119390 rows and 32 columns

### Dataset Information

In [ ]:
# Dataset Info
df.info()

In [ ]:
# Dataset Described
df.describe().T

In [ ]:
# Dataset Columns
df.columns

First of all we will try to understand the meaning of all columns of the dataframe. \
For this we will see the unique values attained by each column whose meaning we are unable to understand.

In [ ]:
df['hotel'].unique()

In [ ]:
df['is_canceled'].unique()

In [ ]:
df['arrival_date_year'].unique()

In [ ]:
df['meal'].unique()

In [ ]:
df['market_segment'].unique()

In [ ]:
df['children'].unique()  # we can see this columns has zero as well as null 

### <b> Cleaning data </b> 

Cleaning data is crucial step before EDA as it will remove the ambigous data that can affect the outcome of EDA.

While cleaning data we will perform following steps: \
1) Remove duplicate rows  \
2) Handling missing values.  \
3) Convert columns to appropriate datatypes.  \
4) Adding impoertant columns

### <b> Step 1: Removing duplicate rows </b>

In [ ]:
# Dataset Duplicate Value Count

len(df)-len(df.drop_duplicates())


there are 31994 duplicate values we need remove these

In [ ]:
new_df = df.drop_duplicates(keep=False)   #dropping duplicates


In [ ]:
new_df.shape   #after deleting the duplicates

### <b> Step2: Handling missing values. </b>

In [ ]:
# Missing Values/Null Values Count
new_df.isnull().sum().sort_values(ascending=False)[:5]

In [ ]:
# Visualizing the missing values
new_df.isnull().sum().sort_values(ascending=False)[:4].plot.bar()


Since, company and agent columns have comany number and agent numbers as data. There may be some cases when customer didnt booked hotel via any agent or via any company. So in that case values can be null under these columns. \
We will replace null values by 0 in these columns 

In [ ]:
#filling null values with zeros 
new_df[['company','agent']] = new_df[['company','agent']].fillna(0)


In [ ]:
#checking unique values in children column
new_df['children'].unique()


This column 'children' has 0 as value which means 0 children were present in group of customers who made that transaction. \
So, 'nan' values are the missing values due to error of recording data. \
 \
We will replace the null values under this column with mean value of children.

In [ ]:
new_df['children'].fillna(new_df['children'].mean(),inplace=True)

In [ ]:
new_df['children'].unique()

Next column with missing value is 'country'.
This column represents the country of origin of customer. \
Since, this column has datatype of string. We will replace the missing value with the mode of 'country' column.

In [ ]:
new_df['country'].mode()

In [ ]:
# mode of country columns is PRT so we would will fill PRT 

new_df['country'].fillna('PRT',inplace=True) 

In [ ]:
# Checking if all null values are removed
new_df.isnull().sum().sort_values(ascending = False)[:6]

Now we have removed all null values

In [ ]:
#Now we will check is there any wrong data 
new_df[new_df['adults']+new_df['babies']+new_df['children']==0].shape

we can see above There are some rows with total number of adults, children or babies equal to zero. So we will remove such rows.



In [ ]:
new_df.drop(new_df[new_df['adults']+new_df['babies']+new_df['children']==0].index,inplace=True) #dropping where total number guest is 0

### <b> Step 3: Converting columns to appropriate datatypes. </b>

In [ ]:
df.dtypes

In [ ]:
# Converting datatype of columns 'children', 'company' and 'agent' from float to int.
new_df[['children', 'company', 'agent']] = new_df[['children','company','agent']].astype('int64')

In [ ]:
# changing datatype of column 'reservation_status_date' from object  to date data_type.
new_df['reservation_status_date'] = pd.to_datetime(new_df['reservation_status_date'],format = '%Y-%m-%d')

### <b> Step 4: Adding important columns. </b>

In [ ]:
# Adding total staying days in hotels
new_df['total_stay'] = new_df['stays_in_weekend_nights']+new_df['stays_in_week_nights']


We are above adding these columns so that we can analyse the stay length at hotels

In [ ]:
# Adding total people num as column, i.e. total people num = num of adults + children + babies
new_df['total_people'] = new_df['adults']+new_df['children']+new_df['babies']

## **EDA** ##

##Now we will see correlation between all numerical variables.

Since, columns like 'is_cancelled', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'is_repeated_guest', 'company', 'agent' are categorical data having numerical type. So we wont need to check them for correlation.\
 \

In [ ]:
#Taking only numerical columns 
num_df1 = new_df[['lead_time','previous_cancellations','previous_bookings_not_canceled','booking_changes','days_in_waiting_list','adr','required_car_parking_spaces','total_of_special_requests','total_stay','total_people']]


In [ ]:
#correlation matrix
corrmat = num_df1.corr()
f, ax = plt.subplots(figsize=(12, 7))
sns.heatmap(corrmat,annot = True,fmt='.2f', annot_kws={'size': 10},  vmax=.8, square=True);

1) Total stay length and lead time have slight correlation. This may means that for longer hotel stays people generally plan little before the the actual arrival.

2) adr is slightly correlated with total_people, which makes sense as more no. of people means more revenue, therefore more adr.

Lets see does length of stay affects the 
adr.

In [ ]:
plt.figure(figsize = (12,6))
sns.scatterplot(y = 'adr', x = 'total_stay', data = new_df)
plt.show()

In [ ]:
#Dropping outlier 
new_df.drop(new_df[new_df['adr'] > 5000].index, inplace = True)

In [ ]:
plt.figure(figsize=(10,6))  
sns.scatterplot(y='adr',x='total_stay',data=new_df)

From scatter plot we can see that as length of  tatal_stay increase the adr decreases this means for longer stay,the better deal for customer can be finalised.

# **Univariate Analysis**

Q1) Which agent makes most no. of bookings?

In [ ]:
d = pd.DataFrame(new_df['agent'].value_counts()).reset_index().rename(columns = {'index':'agent','agent':'num_of_bookings'}).sort_values(by = 'num_of_bookings', ascending = False)
d.drop(d[d['agent'] == 0].index, inplace = True)            # 0 represents that booking is not made by an agent
d = d[:10]                                                   # Selecting top 10 performing agents
plt.figure(figsize = (10,5))
sns.barplot(x = 'agent', y = 'num_of_bookings', data = d, order = d.sort_values('num_of_bookings', ascending = False).agent)

we can see agent 9 made most number of bookings

Q2) Which room type is in most demand and which room type generates highest adr?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,10)) 

sns.countplot(ax = axes[0], x = new_df['assigned_room_type'])
sns.boxplot(ax = axes[1], x = new_df['assigned_room_type'], y = new_df['adr'])
plt.show()


Most demanded room type is A, but better adr rooms are of type H, G and C also. Hotels should increase the no. of room types A and H to maximise revenue.

Q3) Which meal type is most preffered 

In [ ]:
plt.figure(figsize=(12,8))   # set size of the figure
sns.countplot(x=new_df['meal'])

Most preferred meal type is BB (Bed and breakfast).

## <b> (1) Hotel wise analysis </b>

Q1) What is percentage of bookings in each hotel?

In [ ]:
grouped_by_hotel = new_df.groupby('hotel')
d = pd.DataFrame((grouped_by_hotel.size()/new_df.shape[0])*100).reset_index().rename(columns = {0:'Booking %'})      #Calculating percentage
plt.figure(figsize = (8,5))
sns.barplot(x = d['hotel'], y = d['Booking %'] )
plt.show()


Around 60% bookings are for City hotel and 40% bookings are for Resort hotel.

Q2) which hotel seems to make more revenue?

In [ ]:
d1 = grouped_by_hotel['adr'].agg(np.mean).reset_index().rename(columns = {'adr':'avg_adr'})   # calculating average adr
plt.figure(figsize = (8,5))
sns.barplot(x = d1['hotel'], y = d1['avg_adr'] )

Avg adr of Resort hotel is slightly lower than that of City hotel. Hence, City hotel seems to be making slightly more revenue.

Q3) Which hotel has higher lead time?

In [ ]:
d2 = grouped_by_hotel['lead_time'].median().reset_index().rename(columns = {'lead_time':'median_lead_time'})
plt.figure(figsize = (8,5))
sns.barplot(x = d2['hotel'], y = d2['median_lead_time'] )

City hotel has slightly higher median lead time. Also median lead time is significantly higher in each case, this means customers generally plan their hotel visits way to early.

Q4) What is preferred stay length in 
each hotel?

In [ ]:
not_canceled = new_df[new_df['is_canceled'] == 0]
s1 = not_canceled[not_canceled['total_stay'] < 15]
plt.figure(figsize = (10,5))
sns.countplot(x = s1['total_stay'], hue = s1['hotel'])

Most common stay length is less than 4 days and generally people prefer City hotel for short stay, but for long stays, Resort Hotel is preferred.

Q5) Which hotel has longer waiting time?

In [ ]:
d3 = pd.DataFrame(grouped_by_hotel['days_in_waiting_list'].agg(np.mean).reset_index().rename(columns = {'days_in_waiting_list':'avg_waiting_period'}))
plt.figure(figsize = (8,5))
sns.barplot(x = d3['hotel'], y = d3['avg_waiting_period'] )

**City** hotel has significantly longer waiting time, hence City Hotel is much busier than Resort Hotel.

Q6) Which hotel has higher bookings cancellation rate.

In [ ]:
# Selecting and counting number of cancelled bookings for each hotel.
cancelled_data = new_df[new_df['is_canceled'] == 1]
cancel_grp = cancelled_data.groupby('hotel')
D1 = pd.DataFrame(cancel_grp.size()).rename(columns = {0:'total_cancelled_bookings'})

# Counting total number of bookings for each type of hotel
grouped_by_hotel = new_df.groupby('hotel')
total_booking = grouped_by_hotel.size()
D2 = pd.DataFrame(total_booking).rename(columns = {0: 'total_bookings'})
D3 = pd.concat([D1,D2], axis = 1)

# Calculating cancel percentage 
D3['cancel_%'] = round((D3['total_cancelled_bookings']/D3['total_bookings'])*100,2)
D3

In [ ]:
plt.figure(figsize = (10,5))  #Setting Size the figure
sns.barplot(x = D3.index, y = D3['cancel_%'])

we can see city hotel high cancelleation becouse city hotel has almost 28% booking cancellation
and Resort has 22% boooking cancellation

Q7) Which hotel has high chance that its customer will return for another stay?

In [ ]:
#Selecting and counting repeated customers bookings
repeated_data = new_df[new_df['is_repeated_guest']==1]
repeat_grp = repeated_data.groupby('hotel')
D2 = pd.DataFrame(repeat_grp.size()).rename(columns={0:'total_repeated_guests'})

#Counting total bookings
total_booking = grouped_by_hotel.size()
D3 = pd.DataFrame(total_booking).rename(columns= {0:'total_bookings'})
D4 = pd.concat([D2,D3],axis=1)

#Calculating repeat %
D4['repeat_%'] = round((D4['total_repeated_guests']/D4['total_bookings'])*100,2)
D4 = D4.reset_index()
D4

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(x = D4['hotel'], y = D4['repeat_%'])

Both hotels have very small percentage that customer will repeat, but Resort hotel has slightly higher repeat % than City Hotel.

## <b> (2) Distribution Channel wiseAnalysis </b>



Q1) Which is the most common channel for 
booking hotels?

In [ ]:
grouped_by_dist_ch = new_df.groupby('distribution_channel') 
#calculating percentage 
d = pd.DataFrame(round((grouped_by_dist_ch.size()/new_df.shape[0])*100,2)).reset_index().rename(columns={0:'Booking_%'})
plt.figure(figsize = (8,8)) #set size of figure
data = d['Booking_%']
label = d['distribution_channel']
plt.pie(x=data,autopct="%.2f%%",explode=[0.05]*5,labels=label,pctdistance=0.3)
plt.title("Booking % by distribution channels", fontsize=14);


Q2) Which channel is mostly used for early booking of hotels?

In [ ]:
d1 = pd.DataFrame(round(grouped_by_dist_ch['lead_time'].median(),2)).reset_index().rename(columns={'lead_time':'median_lead_time'})
plt.figure(figsize=(7,5))
sns.barplot(x=d1['distribution_channel'],y=d1['median_lead_time'])

TA(Travel Agency)/TO is mostly used for planning Hotel visits ahead of time. But for sudden visits other mediums are most preferred.

Q3) Which channel has longer average 
waiting time?

In [ ]:
d2 = pd.DataFrame(round((grouped_by_dist_ch['days_in_waiting_list']).mean(),2)).reset_index().rename(columns={'days_in_waiting_list':'avg_waiting_time'})
plt.figure(figsize = (7,5))
sns.barplot(x=d2['distribution_channel'],y=d2['avg_waiting_time'])

While booking via TA/TO one may have to wait a little longer to confirm booking of rooms.

Q4) Which distribution channel brings better revenue generating deals for hotels?

In [ ]:
group_by_dc_hotel = new_df.groupby(['distribution_channel', 'hotel'])
d5 = pd.DataFrame(round((group_by_dc_hotel['adr']).agg(np.mean),2)).reset_index().rename(columns = {'adr': 'avg_adr'})
plt.figure(figsize = (7,5))
sns.barplot(x = d5['distribution_channel'], y = d5['avg_adr'], hue = d5['hotel'])
plt.ylim(40,140)
plt.show()

GDS channel brings higher revenue generating deals for City hotel, in contrast to that most bookings come via TA/TO. City Hotel can work to increase outreach on GDS channels to get more higher revenue generating deals.

Resort hotel has more revnue generating deals by direct and TA/TO channel. Resort Hotel need to increase outreach on GDS channel to increase revenue.

## <b> (3) Booking cancellation Analysis </b>

Let us try to understand what causes the people to cancel the booking.

Q1) Which significant distribution channel has highest cancellation percentage?

In [ ]:

d = pd.DataFrame((grouped_by_dist_ch['is_canceled'].sum()/grouped_by_dist_ch.size())*100).drop(index='Undefined').rename(columns={0:'Cancel_%'}) #calculating percentage 
plt.figure(figsize = (10,5))
d = d.reset_index()
sns.barplot(x=d['distribution_channel'],y=d['Cancel_%'])

TA/TO has highest booking cancellation %. Therefore, a booking via TA/TO is 30% likely to get cancelled.

Let us see what causes the cancelation of bookings of rooms by customers \
 \
 One question can arise that may be longer waiting period or longer lead time causes the cancellation of bookings, let us check that.

In [ ]:
waiting_bookings = new_df[new_df['days_in_waiting_list']!=0]  #Selecting Booking with non zero waiting
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
sns.kdeplot(ax=axes[0],x='days_in_waiting_list',hue='is_canceled',data=waiting_bookings)
sns.kdeplot(ax=axes[1], x=new_df['lead_time'],hue=new_df['is_canceled'])

We see that most of the bookings that are cancelled have waiting period of less 150 days but also most of bookings that are not cancelled also have waiting period less than 150 days. Hence this shows that waiting period has no effect on cancellation of bookings. \
\
Also, lead time has no affect on cancellation of bookings, as both curves of cancelation and not cncelation are similar for lead time too.

Now we will check whether not getting allotted the same room type as demanded is the cause of cancellation fo bookings

In [ ]:
#creating a function to check same same allot or not 
def check_room_alloted(x):
  if x['reserved_room_type'] != x['assigned_room_type']:
    return 1
  else:
    return 0
    

In [ ]:
new_df['same_room_not_alloted'] = new_df.apply(lambda x:check_room_alloted(x),axis=1)
group_by_cn = new_df.groupby('is_canceled')
D6 = pd.DataFrame((group_by_cn['same_room_not_alloted'].sum()/group_by_cn.size())*100).rename(columns = {0: 'same_room_not_alloted_%'})
plt.figure(figsize=(10,7))
sns.barplot(x = D6.index, y = D6['same_room_not_alloted_%'])
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(x = 'same_room_not_alloted', y = 'adr', data = new_df)
plt.show()

So not getting same room do affects the adr, people who didn't got same room have paid a little lower adr, except for few exceptions.

## <b> (4) Time wise analysis </b>

(1). In which month most number of  guests are coming ?

In [ ]:
#checking values count in each month
new_df['arrival_date_month'].value_counts()

In [ ]:
#plotting the the countplot
plt.figure(figsize=(16,10))
sns.countplot(x=new_df['arrival_date_month'],hue=new_df['hotel'],order = new_df['arrival_date_month'].value_counts().index)

Most number of guests are coming in august for both city and resort hotel

(2). Which month is producing highest adr.

In [ ]:
reindex = ['January', 'February','March','April','May','June','July','August','September','October','November','December']
new_df['arrival_date_month'] = pd.Categorical(new_df['arrival_date_month'],categories=reindex,ordered=True)
plt.figure(figsize = (15,8))
sns.boxplot(x = new_df['arrival_date_month'],y = new_df['adr'])
plt.show()

Avg adr rises from beginning of year upto middle of year and reaches peak at August and then lowers to the end of year. But hotels do make some good deals with high adr at end of year also.

(3). check the trend of arrival_num and avg adr within a month.

In [ ]:
arr_df = pd.DataFrame(not_canceled.groupby('arrival_date_day_of_month').size()).rename(columns={0:'Arrival_num'})
arr_df['avg_adr'] = not_canceled.groupby('arrival_date_day_of_month')['adr'].agg(np.mean)

In [ ]:
fig ,axes = plt.subplots(1,2,figsize=(18,9))
g = sns.lineplot(ax=axes[0],x=arr_df.index,y=arr_df['Arrival_num'])
g.grid()
g.set_xticks([1,7,14,21,28,31])
g.set_xticklabels([1,7,14,21,28,31])

#plotting avg adr for each day of month
h = sns.lineplot(ax=axes[1],x=arr_df.index,y= arr_df['avg_adr'])
h.grid()
g.set_xticks([1,7,14,21,28,31])
g.set_xticklabels([1,7,14,21,28,31])

We can see that graph Arrival_num has small peaks at regular interval of days. This can be due to increase in arrival weekend.\
\
Also the avg adr tends to go up as month ends. Therefore charge more at the end of month.

Let us divide our customers in three categories of single, couple and family/friends. then check their booking %

In [ ]:
single = not_canceled[(not_canceled['adults']==1) & (not_canceled['children']==0) & (not_canceled['babies']==0)]
couple = not_canceled[(not_canceled['adults']==2) & (not_canceled['children']==0) & (not_canceled['babies']==0)]
family = not_canceled[(not_canceled['adults']==1) + (not_canceled['children']==0) + (not_canceled['babies'] >2)]

reindex = ['January', 'February','March','April','May','June','July','August','September','October','November','December']
fig, ax = plt.subplots(figsize=(12, 8))
for type in ['single','couple','family']:
  d1 = eval(type).groupby(['arrival_date_month']).size().reset_index().rename(columns = {0:'arrival_num'})
  d1['arrival_date_month'] = pd.Categorical(d1['arrival_date_month'],categories=reindex,ordered=True)
  sns.lineplot(data=d1,x='arrival_date_month',y='arrival_num',label=type,ax=ax)


plt.grid()

It is clear from graph that their is a sudden surge in arrival num of couples and family in months of July and August. So better plans can be planned accordingly at that time for these type of customers.

## <b> (5) Region Wise Analysis </b>


(1). From where the most guests are coming ?

In [ ]:
#taking all those country with values count where cancellation is not made
country_wise_guests = new_df[new_df['is_canceled']==0]['country'].value_counts().reset_index().rename(columns={'index':'country','country':'No_of_guests'})
country_wise_guests

In [ ]:
plt.figure(figsize=(10,6))
grouped_by_country = new_df.groupby('country')
#taking top 10 countries with highest number guests
d1 = pd.DataFrame(grouped_by_country.size()).reset_index().rename(columns={0:'Count'}).sort_values('Count',ascending=False)[:10]
sns.barplot(x = d1['country'], y  = d1['Count'])

Most guest are from Portugal and other Europian contries.

# **Conclusion**

(1). Around 60% bookings are for City hotel and 40% bookings are for Resort hotel, therefore City Hotel is busier 
than Resort hotel. Also the overall adr of City hotel is slightly higher than Resort hotel.

(2). Mostly guests stay for less than 5 days in hotel and for longer stays Resort hotel is preferred

(3).Both hotels have significantly higher booking cancellation rates and very few guests less than 3 % return for 

another booking in City hotel. 5% guests return for stay in Resort hotel.
(4). Most of the guests came from european countries, with most no. of guest coming from Portugal.

(5). Guests use different channels for making bookings out of which most preferred way is TA/TO

(6). For hotels higher adr deals come via GDS channel, so hotels should increase their popularity on this channel.

(7).Within a month, adr gradually increases as month ends, with small sudden rise on weekends.

(8). Almost 30% of bookings via TA/TO are cancelled.

(9)July- August are the most busier and profitable months for both of hotels.

(10). Within a month, adr gradually increases as month ends, with small sudden rise on weekends.

(11).For customers, generally the longer stays (more than 15 days) can result in better deals in terms of low adr.

